<a href="https://colab.research.google.com/github/PariSsy/datacamp-notes/blob/main/Python_Writing_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writing Functions in Python

* Instructor = Shayne Miel
* [Course Link](https://learn.datacamp.com/courses/writing-functions-in-python)
* Notes taken = Aug 9, 2021 by [Paris Zhang](https://www.linkedin.com/in/parisyunyuezhang/)



# Chapter 1 - Best Practices

## Section 1.1 - Docstrings

### A complex function

```
def split_and_stack( df, new_names ):
  """
  Split a DataFrame's columns into two halves and then stack them vertically, returning a new DataFrame with `new_names` as the column names.

  Args:
    df (DataFrame): The DataFrame to split.new_names (iterable of str): The column names for the new DataFrame.

  Returns:
    DataFrame
  """
  half = int(len(df.columns) / 2)
  left = df.iloc[:, :half]
  right = df.iloc[:, half:]
  return pd.DataFrame(
    data = np.vstack([left.values, right.values]),
    columns = new_names
  )
```

### Anatomy of a docstring

```
def function_name(arguments):
  """
  Description of what the function does.

  Description of the arguments, if any.

  Description of the return value(s), if any.

  Description of errors raised, if any.

  Optional extra notes or examples of usage.
  """
```

### Docstring formats
* Google Style
* Numpydoc
* reStructuredText
* EpyText

#### Google style - arguments & return value(s)

```
def function(arg_1, arg_2=42):
  """Description of what the function does.

  Args:
    arg_1 (str): Description of arg_1 that can break onto the next line if needed.
    arg_2 (int, optional): Write optional when an argument has a default value.
  
  Returns:
    bool: Optional description of the return value
    Extra lines are not indented.
  
  Raises:
    ValueError: Include any error types that the function intentionally raises.
  
  Notes:
    See https://www.datacamp.com/community/tutorials/docstrings-python for more info.
```

#### Numpydoc

```
def function(arg_1, arg_2=42):
  """
  Description of what the function does.

  Parameters
  ----------
  arg_1 : expected type of arg_1
    Description of arg_1.
  arg_2 : int, optional
    Write optional when an argument has a default value.
    Default = 42.
  
  Returns
  -------
  The type of the return value
    Can include a description of the return value.
    Replace "Returns" with "Yields" if this function is a generator.
  """
```

### Retrieving docstrings

```
def the_answer():
  """Return the answer to life, the universe, and everything.

  Returns:
    int
  """
  return 42
print(the_answer.__doc__)

>>> Return the answer to life, the universe, and everything.
>>>
>>>  Returns:
>>>    int
```

```
import insepct
print(inspect.getdoc(the_answer))

>>> Return the answer to life, the universe, and everything.
>>>
>>>  Returns:
>>>    int
```

### Examples

1. Count the number of times the letter appears in the string:

```
def count_letter(content, letter):
  """Count the number of times `letter` appears in `content`.

  Args:
    content (str): The string to search.
    letter (str): The letter to search for.

  Returns:
    int

  # Add a section detailing what errors might be raised
  Raises:
    ValueError: If `letter` is not a one-character string.
  """
  if (not isinstance(letter, str)) or len(letter) != 1:
    raise ValueError('`letter` must be a single character string.')
  return len([char for char in content if char == letter])
```

2. A feature that displays a tooltip with a function's docstring whenever the user starts typing the function name.

```
import inspect

def build_tooltip(function):
  """Create a tooltip for any function that shows the
  function's docstring.

  Args:
    function (callable): The function we want a tooltip for.

  Returns:
    str
  """
  # Get the docstring for the "function" argument by using inspect
  docstring = inspect.getdoc(function)
  border = '#' * 28
  return '{}\n{}\n{}'.format(border, docstring, border)

print(build_tooltip(count_letter))
print(build_tooltip(range))
print(build_tooltip(print))
```

## Section 1.2 - DRY and "Do One Thing"

### Don't repeat yourself (DRY)

* Training set
```
train = pd.read_csv('train.csv')
train_y = train['labels'].values
train_X = train[col for col in train.columns if col != 'labels'].values
train_pca = PCA(n_components=2).fit_transform(train_X)
plt.scatter(train_pca[:,0], train_pca[:,1])
```
* Validation set
```
val = pd.read_csv('validation.csv')
val_y = val['labels'].values
val_X = val[col for col in val.columns if col != 'labels'].values
val_pca = PCA(n_components=2).fit_transform(val_X)
plt.scatter(val_pca[:,0], val_pca[:,1])
```
* Testing set
```
test = pd.read_csv('test.csv')
test_y = test['labels'].values
test_X = test[col for col in test.columns if col != 'labels'].values
test_pca = PCA(n_components=2).fit_transform(test_X)
plt.scatter(test_pca[:,0], test_pca[:,1])
```

### Use functions to avoid repetition

```
def load_and_plot(path):
  """Load a data set and plot the first two principal components.

  Args:
    path (str): The location of a CSV file.
  
  Returns:
    tuple of ndarray: (features, labels)
  """
  data = pd.read_csv(path)
  y = data['label'].values
  X = data[col for col in train.columns if col != 'label'].values
  pca = PCA(n_components=2).fit_transform(X)
  plt.scatter(pca[:,0], pca[:,1])
  return X,y

train_X, train_y = load_and_plot('train.csv')
val_X, val_y = load_and_plot('val.csv')
test_X, test_y = load_and_plot('test.csv')
```

### Do One Thing

```
def load_data(path):
  """Load a data set.

  Args:
    path (str): The location of a CSV file.

  Returns:
    tuple of ndarray: (features, labels)
  """
  data = pd.read_csv(path)
  y = data['labels'].values
  X = data[col for col in data.columns if col != 'labels'].values
  return X, y
```

```
def plot_data():
  """Plot the first two principal components of a matrix.

  Args:
    X (numpy.ndarray): The data to plot.
  """
  pca = PCA(n_components=2).fit_transform(X)
  plt.scatter(pca[:,0], pca[:,1])
```

**Advantages**:
1. More flexible
2. More easily understood
3. Simpler to test
4. Simpler to debug
5. Easier to change

## Section 1.3 - Pass by assignment

### Example 1

```
def foo(x):
  x[0] = 99
my_list = [1,2,3]
foo(my_list)
print(my_list)

>>> [99,2,3]

def bar(x):
  x = x + 90
my_var = 3
bar(my_var)
print(my_var)

>>> 3
```

### Example 2

```
a = [1, 2, 3]
b = a
a.append(4)
print(b)

>>> [1, 2, 3, 4]

b.append(5)
print(a)

>>> [1, 2, 3, 4, 5]
```

### Immutable
* int
* float
* bool
* string
* bytes
* tuple
* frozenset
* None

### Mutable
* list
* dict
* set
* bytearray
* objects
* functions
* almost everything else

### Example where mutable default arguments are dangerous

```
def foo(var=[]):
  var.append(1)
  return var
foo()

>>> [1]

foo()

>>> [1, 1]
```

```
def foo(var=None):
  if var is None:
    var = []
  var.append(1)
  return var
foo()

>>> [1]

foo()

>>> [1]
```

### Exercise 1

```
def store_lower(_dict, _string):
  """Add a mapping between `_string` and a lowercased version of `_string` to `_dict`

  Args:
    _dict (dict): The dictionary to update.
    _string (str): The string to add.
  """
  orig_string = _string
  _string = _string.lower()
  _dict[orig_string] = _string

d = {}
s = 'Hello'

store_lower(d, s)

>>> d = {'Hello': 'hello'}
>>> s = 'Hello'
```

# Chapter 2 - Context Managers



## Section 2.1 - Using context managers

## Section 2.2 - Writing context managers

## Section 2.3 - Advanced topics

# Chapter 3 - Decorators



## Section 3.1 - Functions are objects

## Section 3.2 - Scope

## Section 3.3 - Closures

## Section 3.4 - Decorators

# Chapter 4 - More on Decorators


## Section 4.1 - Examples

## Section 4.2 - Decorators and metadata

## Section 4.3 - Decorators that take arguments

## Section 4.4 - Timeout() example